In [7]:
import numpy as np
from hog36 import hog36
import math 
import os
import cv2
import random
from glob import glob
from logistic_prob import logistic_prob

In [8]:
cd C:\Users\Matt Hetrick\Desktop\COS429\Assignment 2\a2_part3_starter

C:\Users\Matt Hetrick\Desktop\COS429\Assignment 2\a2_part3_starter


In [9]:
import os
import cv2
import random
from glob import glob
import numpy as np
import math 
from hog36 import hog36


def get_training_data(n, orientations, wrap180):
    """Reads in examples of faces and nonfaces, and builds a matrix of HoG
       descriptors, ready to pass in to logistic_fit

    Args:
        n: nu2mber of face and nonface training examples (n of each)
        orientations: the number of HoG gradient orientations to use
        wrap180: if true, the HoG orientations cover 180 degrees, else 360

    Returns:
        descriptors: matrix of descriptors for all 2*n training examples, where
                     each row contains the HoG descriptor for one face or nonface
        classes: vector indicating whether each example is a face (1) or nonface (0)
    """
    training_faces_dir = 'face_data/training_faces'
    training_nonfaces_dir = 'face_data/training_nonfaces'
    hog_input_size = 36
    hog_descriptor_size = 100 * orientations

    # Get the names of the first n training faces
    face_filenames = sorted(glob(os.path.join(training_faces_dir, '*.jpg')))
    num_face_filenames = len(face_filenames)
    if num_face_filenames > n:
        face_filenames = face_filenames[:n]
    elif num_face_filenames < n:
        n = num_face_filenames

    # Initialize descriptors, classes
    descriptors = np.zeros([2 * n, hog_descriptor_size + 1])
    classes = np.zeros([2 * n])

    # Loop over faces
    for i in range(n):
        # Read the next face file
        face = cv2.imread(face_filenames[i], cv2.IMREAD_GRAYSCALE)

        # Compute HoG descriptor
        face_descriptor = hog36(face, orientations, wrap180)

        # Fill in descriptors and classes
        descriptors[i, 0] = 1
        descriptors[i, 1:] = face_descriptor
        classes[i] = 1

    # Get the names of the nonfaces
    nonface_filenames = sorted(glob(os.path.join(training_nonfaces_dir, '*.jpg')))
    num_nonface_filenames = len(nonface_filenames)

    # Loop over all nonface samples we want
    for i in range(n, 2 * n):
        # Read a random nonface file
        j = random.randint(0, num_nonface_filenames - 1)
        nonface = cv2.imread(nonface_filenames[j], cv2.IMREAD_GRAYSCALE)
 
        # random numbers for random cropping :: crop set up 
        height = nonface.shape[0]
        width = nonface.shape[1]   
        split = hog_input_size/2
        maxEdge = min(height-1,width-1)
        # put -2 so we dont pick a corner pixel for convenience of boundary cases
        # need a size of at least 36 so that resize doesnt fail so location cannot be under 18 on either side
        location = random.randint(split, maxEdge-split)
        # loc+size <= min(h,w) and loc-size >= 0. Solve to get loc-min(h,w) <= size <= loc
        size = random.randint(split, min(math.fabs(0-location), maxEdge - location))
    
        # Crop out a random square at least hog_input_size
        # random size square 
        crop = nonface[location-size:location+size,location-size:location+size] 

        # Resize to be the right size
        crop = cv2.resize(crop, (hog_input_size, hog_input_size))
        # Compute HoG descriptor
        non_face_descriptor = hog36(crop, orientations, wrap180)

        # Fill in descriptors and classes
        descriptors[i, 0] = 1
        descriptors[i, 1:] = non_face_descriptor
        classes[i] = 0

    return descriptors, classes

In [12]:
import numpy as np
#from get_training_data import get_training_data
#from get_testing_data import get_testing_data
from logistic_prob import logistic_prob
from logistic_fit import logistic_fit
import matplotlib.pyplot as plt


def test_face_classifier(ntrain, ntest, orientations, wrap180):
    """Train and test a face classifier.

    Args:
        ntrain: number of face and nonface training examples (ntrain of each)
        ntest: number of face and nonface testing examples (ntest of each)
        orientations: the number of HoG gradient orientations to use
        wrap180: if true, the HoG orientations cover 180 degrees, else 360
    """
    # Get some training data
    descriptors, classes = get_training_data(ntrain, orientations, wrap180)

    # Train a classifier
    params = logistic_fit(descriptors, classes, 0.001)

    # Evaluate the classifier on the training data
    predicted = logistic_prob(descriptors, params)
    plot_errors(predicted, classes, 'Performance on training set for varying threshold', 1)

    # Get some test data
    tdescriptors, tclasses = get_testing_data(ntest, orientations, wrap180)

    # Evaluate the classifier on the test data
    tpredicted = logistic_prob(tdescriptors, params)
    plot_errors(tpredicted, tclasses, 'Performance on test set for varying threshold', 2)


def plot_errors(predicted, classes, name, num):
    """Plot a log/log graph of miss rate (false negatives) vs false positives
       for a variety of thresholds on probability.

    Args:
        predicted: probabilities that the class is 1
        classes: ground-truth class labels (0/1)
        name: name of the figure
        num: number of the figure
    """
    nthresh = 99
    npts = predicted.shape[0]

    falsepos = np.zeros([nthresh])
    falseneg = np.zeros([nthresh])

    stepsize = 1. / (nthresh + 1)
    for i in range(nthresh):
        thresh = (i + 1) * stepsize
        falsepos[i] = np.sum(np.logical_and(predicted >= thresh, classes == 0)) / npts
        falseneg[i] = np.sum(np.logical_and(predicted < thresh, classes == 1)) / npts

    limit = 1e-4
    plt.figure(num)
    plt.title(name)
    plt.loglog(np.maximum(falsepos, limit), np.maximum(falseneg, limit))
    plt.axis([limit, 1, limit, 1])
    plt.xlabel('False positive rate')
    plt.ylabel('False negative rate')

    plt.show()
    
    np.savez('face_classifier.npz', params = params, orientations=orientations, wrap180=wrap180)

In [13]:
def find_faces_single_scale(img, stride, thresh, params, orientations, wrap180):
    """Find 36x36 faces in an image

    Args:
        img: an image
        stride: how far to move between locations at which the detector is run
        thresh: probability threshold for calling a detection a face
        params: trained face classifier parameters
        orientations: the number of HoG gradient orientations to use
        wrap180: if true, the HoG orientations cover 180 degrees, else 360

    Returns:
        outimg: copy of img with face locations marked
        probmap: probability map of face detections
    """
    windowsize = 36
    if stride > windowsize:
        stride = windowsize

    height, width = img.shape
    probmap = np.zeros([height, width])
    outimg = np.array(img)
    
    new_params = np.zeros([hog_descriptor_size + 1, 1])
    new_params[:,0] = params
    params = new_params

    # Loop over windowsize x windowsize windows, advancing by stride
    hog_descriptor_size = 100 * orientations
    window_descriptor = np.zeros([1, hog_descriptor_size + 1])

    # // slides down then across the image, by stride
    for i in range(0, width-windowsize, stride):
        for j in range(0, height-windowsize, stride):

            # Crop out a windowsize x windowsize window starting at (i,j)
            crop = img[i:i+windowsize,j:j+windowsize] 

            # Compute a HoG descriptor, and run the classifier
            window_descriptor[0,0] = 1
            window_descriptor[0, 1:] = hog36(crop, orientations, wrap180)
            # NEED TO TRAIN AND RUN CLASSIFIER ?? PROB --> FIT () ?? or since trained params good 
            probability = logistic_prob1(window_descriptor, params) #or need to do both fit + prob 

            # Mark detection probability in probmap
            win_i = i + int((windowsize - stride) / 2)
            win_j = j + int((windowsize - stride) / 2)
            probmap[win_i:win_i+stride, win_j:win_j+stride] = probability

            # If probability of a face is below thresh, continue 
            # else mark the face on img 
            if probability < thresh:
                continue
             
            #print("got here")
            # Mark the face in outimg
            outimg[i, j:j+windowsize] = 255
            outimg[i+windowsize-1, j:j+windowsize] = 255
            outimg[i:i+windowsize, j] = 255
            outimg[i:i+windowsize, j+windowsize-1] = 255

    return outimg, probmap

In [14]:
import matplotlib.pyplot as plt
from find_faces_single_scale import find_faces_single_scale
img = cv2.imread('face_data/single_scale_scenes/addams-family.jpg', cv2.IMREAD_GRAYSCALE)
saved = np.load('face_classifier.npz')
params, orientations, wrap180 = saved['params'], saved['orientations'], saved['wrap180']
outimg, probmap = find_faces_single_scale(img, 3, 0.95, params, orientations, wrap180)
plt.figure(1)
plt.title('outimg')
plt.imshow(outimg, cmap='gray')
plt.show()
plt.figure(2)
plt.title('probmap')
plt.imshow(probmap, cmap='gray')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'face_classifier.npz'